In [2]:
#XG Boost Regression
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, acf
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm

df = pd.read_csv('data/final_data_day.csv')
df_15 = pd.read_csv('data/final_data_15min.csv')

/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_17313/3575399629.py:16: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_15 = pd.read_csv('data/final_data_15min.csv')


In [3]:
df

,DATE,ATTRACTION NAME,WAIT TIME,NB UNITS,GUEST CARRIED,CAPACITY,ADJUST CAPACITY,OPEN_TIME,REAL TIME ATTRACTION OPEN FOR 15MIN,REAL TIME ATTRACTION CLOSES FOR 15MIN,...,TEMPERATURE,TEMPERATURE FELT,TEMPERATURE MINIMUM,TEMPERATURE MAXIMUM,PRESSURE,HUMIDITY,WIND SPEED,WIND DEG,CLOUD PERCENTAGE,ATTENDANCE
0,2018-01-01,Bumper Cars,4.732143,18.000000,128.500004,219.872652,219.873513,12.946429,12.946429,0.000000,...,7.496429,3.279286,7.027143,8.045714,1002.928571,86.285714,8.543571,240.571429,75.714286,0.0
1,2018-01-01,Zipline,10.000000,11.125000,36.491071,92.209821,79.536607,13.660714,12.732143,0.928571,...,7.496429,3.279286,7.027143,8.045714,1002.928571,86.285714,8.543571,240.571429,75.714286,0.0
2,2018-01-01,Water Ride,15.535714,10.010714,122.321425,205.834161,186.788696,12.500000,12.500000,0.000000,...,7.496429,3.279286,7.027143,8.045714,1002.928571,86.285714,8.543571,240.571429,75.714286,0.0
3,2018-01-01,Swing Ride,34.196429,11.976191,94.705357,211.680357,203.123214,13.107143,12.607143,0.500000,...,7.496429,3.279286,7.027143,8.045714,1002.928571,86.285714,8.543571,240.571429,75.714286,0.0
4,2018-01-01,Superman Ride,12.500000,2.177380,38.857138,128.549107,112.491661,7.910714,7.910714,0.000000,...,7.496429,3.279286,7.027143,8.045714,1002.928571,86.285714,8.543571,240.571429,75.714286,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42335,2022-08-18,Bungee Jump,19.375000,4.611904,198.625000,295.794643,271.940179,14.464286,14.464286,0.000000,...,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.225420,70.637711,0.0
42336,2022-08-18,Bumper Cars,7.946429,18.000000,160.178571,231.700286,231.701196,13.642857,13.642857,0.000000,...,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.225420,70.637711,0.0
42337,2022-08-18,Water Ride,23.125000,9.000000,95.785714,226.123527,187.896421,13.732143,11.410714,2.321429,...,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.225420,70.637711,0.0
42338,2022-08-18,Haunted House,18.750000,8.885714,115.910714,204.107143,201.250000,13.607143,13.607143,0.000000,...,7.534682,5.375491,6.569751,8.556166,1017.159478,83.524512,4.237857,185.225420,70.637711,0.0


In [4]:
df_15['DATE'] = pd.to_datetime(df_15['DATE'])

# Filter rows where the year is greater than 2020
df_15_bis = df_15[df_15['DATE'].dt.year > 2020]
df_15_bis = df_15_bis[df_15_bis['ATTENDANCE'] > 0]
df_15_bis = df_15_bis[df_15_bis['GUEST CARRIED'] > 0]
df_15_bis.sort_values('DATE', inplace = True)
df_15_bis.drop(columns = ['START OF 1 RIDE FOR ATTRACTION','END OF 1 RIDE FOR ATTRACTION', 'ATTRACTION'], inplace = True)


In [5]:
"""import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error

# Assuming df_15_bis is your DataFrame
available_attractions = df_15_bis['ATTRACTION NAME'].unique()

app = dash.Dash(__name__)

# Set a white background and use external CSS for better styling
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div([
    html.H1("Attraction Wait Time Prediction", style={'textAlign': 'center'}),
    html.Div([
        dcc.Dropdown(
            id='attraction-dropdown',
            options=[{'label': i, 'value': i} for i in available_attractions],
            value='Dizzy Dropper',  # Set a default value
            style={'width': '50%', 'display': 'inline-block'}
        ),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=df_15_bis['DATE'].min(),
            end_date=df_15_bis['DATE'].max(),
            display_format='YYYY-MM-DD',
            style={'display': 'inline-block'}
        )
    ], style={'textAlign': 'center', 'backgroundColor': 'white'}),
    dcc.Graph(id='prediction-plot'),
    html.Div(id='mse-output', style={'textAlign': 'center', 'fontSize': 20, 'backgroundColor': 'white'})
], style={'width': '80%', 'margin-left': 'auto', 'margin-right': 'auto', 'backgroundColor': 'white'})

@app.callback(
    [Output('prediction-plot', 'figure'),
     Output('mse-output', 'children')],
    [Input('attraction-dropdown', 'value'),  # Écoute les changements de valeur dans le dropdown
     Input('date-picker-range', 'start_date'),  # Écoute les changements de la date de début
     Input('date-picker-range', 'end_date')]  # Écoute les changements de la date de fin
)
def update_output(selected_attraction, start_date, end_date):
    # Filter the dataset for the selected attraction and date range
    filtered_df = df[(df['ATTRACTION NAME'] == selected_attraction) & 
                     (df['DATE'] >= start_date) & 
                     (df['DATE'] <= end_date)]

    # Assuming 'DATE' is the index
    filtered_df = filtered_df.reset_index().rename(columns={'DATE': 'ds', 'WAIT TIME': 'y'})

    # Initialize and fit the Prophet model
    model = Prophet(weekly_seasonality=True, daily_seasonality=True, yearly_seasonality=True)
    model.fit(filtered_df)

    # Make future predictions; you might need to adjust this part based on your actual date range
    future_dates = model.make_future_dataframe(periods=365)
    forecast = model.predict(future_dates)

    # Assuming you have actual values to compare against in `filtered_df`
    # This is a simplification; you'll likely need to adjust the test set creation
    actuals = filtered_df['y'][-90:]  # This assumes you have actual values for the last year
    predictions = forecast['yhat'][-90:]
    
    # Calculate MSE
    mse = mean_squared_error(actuals, predictions)

    # Prepare the plotly figure
    figure = {
        'data': [
            {'x': forecast['ds'], 'y': forecast['yhat'], 'type': 'line', 'name': 'Predicted'},
            {'x': filtered_df['ds'], 'y': filtered_df['y'], 'type': 'line', 'name': 'Actual'},
        ],
        'layout': {
            'title': 'Predictions vs Actuals',
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'WAIT TIME'},
        }
    }

    # Return the figure and MSE text
    return figure, f'MSE: {mse}'

if __name__ == '__main__':
    app.run_server(debug=True)
"""

'import dash\nfrom dash import dcc, html\nfrom dash.dependencies import Input, Output\nimport plotly.graph_objs as go\nimport pandas as pd\nfrom prophet import Prophet\nfrom sklearn.metrics import mean_squared_error\n\n# Assuming df_15_bis is your DataFrame\navailable_attractions = df_15_bis[\'ATTRACTION NAME\'].unique()\n\napp = dash.Dash(__name__)\n\n# Set a white background and use external CSS for better styling\napp.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})\n\napp.layout = html.Div([\n    html.H1("Attraction Wait Time Prediction", style={\'textAlign\': \'center\'}),\n    html.Div([\n        dcc.Dropdown(\n            id=\'attraction-dropdown\',\n            options=[{\'label\': i, \'value\': i} for i in available_attractions],\n            value=\'Dizzy Dropper\',  # Set a default value\n            style={\'width\': \'50%\', \'display\': \'inline-block\'}\n        ),\n        dcc.DatePickerRange(\n            id=\'date-picker-range\',\n       

In [6]:
"""import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming df_15_bis is your DataFrame
available_attractions = df_15_bis['ATTRACTION NAME'].unique()

app = dash.Dash(__name__)

# External CSS for better styling
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div([
    html.H1("Attraction Wait Time Prediction", style={'textAlign': 'center'}),
    html.Div([
        dcc.Dropdown(
            id='attraction-dropdown',
            options=[{'label': i, 'value': i} for i in available_attractions],
            value='Dizzy Dropper',  # Default value
            style={'width': '50%', 'display': 'inline-block'}
        ),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=df_15_bis['DATE'].min(),
            end_date=df_15_bis['DATE'].max(),
            display_format='YYYY-MM-DD',
            style={'display': 'inline-block'}
        )
    ], style={'textAlign': 'center', 'backgroundColor': 'white'}),
    dcc.Graph(id='prediction-plot'),
    dcc.Graph(id='heatmap-plot'),  # ADDITION: New Graph for heatmap
    html.Div(id='mse-output', style={'textAlign': 'center', 'fontSize': 20, 'backgroundColor': 'white'})
], style={'width': '80%', 'margin-left': 'auto', 'margin-right': 'auto', 'backgroundColor': 'white'})

@app.callback(
    [Output('prediction-plot', 'figure'),
     Output('heatmap-plot', 'figure'),
     Output('mse-output', 'children')],
    [Input('attraction-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_output(selected_attraction, start_date, end_date):
    # Filter the dataset for the selected attraction and date range
    filtered_df = df[(df['ATTRACTION NAME'] == selected_attraction) & 
                            (df['DATE'] >= start_date) & 
                            (df['DATE'] <= end_date)]
                            
    # Convert 'DATE' to datetime, if not already
    filtered_df['DATE'] = pd.to_datetime(filtered_df['DATE'])
    
    # Preparing data for Prophet
    prophet_df = filtered_df[['DATE', 'WAIT TIME']].rename(columns={'DATE': 'ds', 'WAIT TIME': 'y'})

    # Initialize and fit the Prophet model
    model = Prophet(weekly_seasonality=True, daily_seasonality=True, yearly_seasonality=True)
    model.fit(prophet_df)

    # Make future predictions; you might need to adjust this part based on your actual date range
    future_dates = model.make_future_dataframe(periods=365)
    forecast = model.predict(future_dates)

    # Prepare the plotly figure for predictions
    prediction_fig = go.Figure()
    prediction_fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicted'))
    prediction_fig.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['y'], mode='lines', name='Actual'))
    prediction_fig.update_layout(title='Predictions vs Actuals', xaxis_title='Date', yaxis_title='Wait Time')

    # Heatmap Data Preparation
    filtered_df['day_of_week'] = filtered_df['DATE'].dt.dayofweek
    filtered_df['day_of_week'] =  filtered_df['day_of_week'].sort_values()
    filtered_df['day_of_week'] = filtered_df['day_of_week'].replace({0:'Monday', 
                                                                     1: 'Tuesday', 
                                                                     2: 'Wednesday', 
                                                                     3: 'Thurday', 
                                                                     4: 'Friday', 
                                                                     5: 'Saturday', 
                                                                     6: 'Sunday'})
    heatmap_df = filtered_df.pivot_table(index='day_of_week', columns='HOUR START OF 1 RIDE', values='WAIT TIME', aggfunc=np.mean)
    heatmap_fig = px.imshow(heatmap_df, labels=dict(x="Hour of Start", y="Day of Week", color="Average Wait Time"),
                            aspect="auto", color_continuous_scale='Viridis')
    heatmap_fig.update_layout(title='Average Wait Time Heatmap')

    # Placeholder for MSE calculation
    mse_text = "MSE calculation placeholder"  # Adjust this to calculate MSE if needed

    return prediction_fig, heatmap_fig, mse_text


if __name__ == '__main__':
    app.run_server(debug=True)
"""

'import dash\nfrom dash import dcc, html\nfrom dash.dependencies import Input, Output\nimport plotly.graph_objs as go\nimport plotly.express as px\nimport pandas as pd\nfrom prophet import Prophet\nfrom sklearn.metrics import mean_squared_error\nimport numpy as np\n\n# Assuming df_15_bis is your DataFrame\navailable_attractions = df_15_bis[\'ATTRACTION NAME\'].unique()\n\napp = dash.Dash(__name__)\n\n# External CSS for better styling\napp.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})\n\napp.layout = html.Div([\n    html.H1("Attraction Wait Time Prediction", style={\'textAlign\': \'center\'}),\n    html.Div([\n        dcc.Dropdown(\n            id=\'attraction-dropdown\',\n            options=[{\'label\': i, \'value\': i} for i in available_attractions],\n            value=\'Dizzy Dropper\',  # Default value\n            style={\'width\': \'50%\', \'display\': \'inline-block\'}\n        ),\n        dcc.DatePickerRange(\n            id=\'date-picker-range

In [7]:
"""import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.express as px
from prophet import Prophet
from sklearn.metrics import mean_squared_error

# Assuming df_15_bis is your DataFrame
available_attractions = df_15_bis['ATTRACTION NAME'].unique()

app = dash.Dash(__name__)

# Set a white background and use external CSS for better styling
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div([
    html.H1("Attraction Wait Time Prediction", style={'textAlign': 'center'}),
    html.Div([
        dcc.Dropdown(
            id='attraction-dropdown',
            options=[{'label': i, 'value': i} for i in available_attractions],
            value='Dizzy Dropper',  # Set a default value
            style={'width': '50%', 'display': 'inline-block'}
        ),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=df_15_bis['DATE'].min(),
            end_date=df_15_bis['DATE'].max(),
            display_format='YYYY-MM-DD',
            style={'display': 'inline-block'}
        )
    ], style={'textAlign': 'center', 'backgroundColor': 'white'}),
    dcc.Graph(id='prediction-plot'),
    html.Div(id='mse-output', style={'textAlign': 'center', 'fontSize': 20, 'backgroundColor': 'white'}),
    dcc.Graph(id='heatmap-plot'),  # New Graph component for the heatmap
], style={'width': '80%', 'margin-left': 'auto', 'margin-right': 'auto', 'backgroundColor': 'white'})

@app.callback(
    [Output('prediction-plot', 'figure'),
     Output('mse-output', 'children'),
     Output('heatmap-plot', 'figure')],  # Adding Output for the heatmap
    [Input('attraction-dropdown', 'value'),  # Listening to changes in dropdown value
     Input('date-picker-range', 'start_date'),  # Listening to changes in start date
     Input('date-picker-range', 'end_date')]  # Listening to changes in end date
)
def update_output(selected_attraction, start_date, end_date):
    # Filter the dataset for the selected attraction and date range
    filtered_df = df[(df['ATTRACTION NAME'] == selected_attraction) & 
                     (df['DATE'] >= start_date) & 
                     (df['DATE'] <= end_date) ]

    filtered_df_15 = df_15_bis[(df_15_bis['ATTRACTION NAME'] == selected_attraction) &
                                 (df_15_bis['DATE'] >= start_date) &
                                    (df_15_bis['DATE'] <= end_date)]

    # Assuming 'DATE' is the index
    filtered_df = filtered_df.reset_index().rename(columns={'DATE': 'ds', 'WAIT TIME': 'y'})


    # Initialize and fit the Prophet model
    model = Prophet(weekly_seasonality=True, daily_seasonality=True, yearly_seasonality=True)
    model.fit(filtered_df)

    # Make future predictions
    future_dates = model.make_future_dataframe(periods=365)
    forecast = model.predict(future_dates)

    # Calculate MSE for the last 90 days as an example
    actuals = filtered_df['y'][-90:]
    predictions = forecast['yhat'][-90:]
    mse = mean_squared_error(actuals, predictions)

    # Prepare the plotly figure for predictions
    figure = {
        'data': [
            {'x': forecast['ds'], 'y': forecast['yhat'], 'type': 'line', 'name': 'Predicted'},
            {'x': filtered_df['ds'], 'y': filtered_df['y'], 'type': 'line', 'name': 'Actual'},
        ],
        'layout': {
            'title': 'Predictions vs Actuals',
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'WAIT TIME'},
        }
    }

    # Heatmap Data Preparation
    filtered_df_15['DATE'] = pd.to_datetime(filtered_df_15['DATE'])  # Ensure 'ds' is datetime for dayofweek
    filtered_df_15['day_of_week'] = filtered_df_15['DATE'].dt.dayofweek
    filtered_df_15['day_of_week'] = filtered_df_15['day_of_week'].sort_values()
    filtered_df_15['day_of_week'] = filtered_df_15['day_of_week'].replace({
        0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
        4: 'Friday', 5: 'Saturday', 6: 'Sunday'
    })
    heatmap_df = filtered_df_15.pivot_table(index='day_of_week', columns='HOUR START OF 1 RIDE', values='WAIT TIME', aggfunc=np.mean)
    heatmap_fig = px.imshow(heatmap_df, labels=dict(x="Hour of Start", y="Day of Week", color="Average Wait Time"),
                            aspect="auto", color_continuous_scale='Viridis')
    heatmap_fig.update_layout(title='Average Wait Time Heatmap')

    # Return the figures and MSE text
    return figure, f'MSE: {mse}', heatmap_fig

if __name__ == '__main__':
    app.run_server(debug=True)
"""

'import dash\nfrom dash import dcc, html\nfrom dash.dependencies import Input, Output\nimport plotly.graph_objs as go\nimport pandas as pd\nimport numpy as np\nimport plotly.express as px\nfrom prophet import Prophet\nfrom sklearn.metrics import mean_squared_error\n\n# Assuming df_15_bis is your DataFrame\navailable_attractions = df_15_bis[\'ATTRACTION NAME\'].unique()\n\napp = dash.Dash(__name__)\n\n# Set a white background and use external CSS for better styling\napp.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})\n\napp.layout = html.Div([\n    html.H1("Attraction Wait Time Prediction", style={\'textAlign\': \'center\'}),\n    html.Div([\n        dcc.Dropdown(\n            id=\'attraction-dropdown\',\n            options=[{\'label\': i, \'value\': i} for i in available_attractions],\n            value=\'Dizzy Dropper\',  # Set a default value\n            style={\'width\': \'50%\', \'display\': \'inline-block\'}\n        ),\n        dcc.DatePickerRange

In [8]:
"""import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import plotly.express as px
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, acf
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm

df = pd.read_csv('data/final_data_day.csv')
df_15 = pd.read_csv('data/final_data_15min.csv')

# Assuming df_15_bis is your DataFrame
available_attractions = df_15_bis['ATTRACTION NAME'].unique()

app = dash.Dash(__name__)

# Set a white background and use external CSS for better styling
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

app.layout = html.Div([
    html.H1("Attraction Wait Time Prediction", style={'textAlign': 'center'}),
    html.Div([
        dcc.Dropdown(
            id='attraction-dropdown',
            options=[{'label': i, 'value': i} for i in available_attractions],
            value='Dizzy Dropper',  # Set a default value
            style={'width': '50%', 'display': 'inline-block'}
        ),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date=df_15_bis['DATE'].min(),
            end_date=df_15_bis['DATE'].max(),
            display_format='YYYY-MM-DD',
            style={'display': 'inline-block'}
        )
    ], style={'textAlign': 'center', 'backgroundColor': 'white'}),
    dcc.Graph(id='prediction-plot'),
    html.Div(id='mse-output', style={'textAlign': 'center', 'fontSize': 20, 'backgroundColor': 'white'}),
    dcc.Graph(id='heatmap-plot'),  # New Graph component for the heatmap
], style={'width': '80%', 'margin-left': 'auto', 'margin-right': 'auto', 'backgroundColor': 'white'})

@app.callback(
    [Output('prediction-plot', 'figure'),
     Output('mse-output', 'children'),
     Output('heatmap-plot', 'figure')],  # Adding Output for the heatmap
    [Input('attraction-dropdown', 'value'),  # Listening to changes in dropdown value
     Input('date-picker-range', 'start_date'),  # Listening to changes in start date
     Input('date-picker-range', 'end_date')]  # Listening to changes in end date
)
def update_output(selected_attraction, start_date, end_date):
    # Filter the dataset for the selected attraction and date range
    filtered_df = df[(df['ATTRACTION NAME'] == selected_attraction) & 
                     (df['DATE'] >= start_date) & 
                     (df['DATE'] <= end_date) ]

    filtered_df_15 = df_15_bis[(df_15_bis['ATTRACTION NAME'] == selected_attraction) &
                                 (df_15_bis['DATE'] >= start_date) &
                                    (df_15_bis['DATE'] <= end_date)]

    # Assuming 'DATE' is the index
    filtered_df = filtered_df.reset_index().rename(columns={'DATE': 'ds', 'WAIT TIME': 'y'})


    # Initialize and fit the Prophet model
    model = Prophet(weekly_seasonality=True, daily_seasonality=True, yearly_seasonality=True)
    model.fit(filtered_df)

    # Make future predictions
    future_dates = model.make_future_dataframe(periods=365)
    forecast = model.predict(future_dates)

    # Calculate MSE for the last 90 days as an example
    actuals = filtered_df['y'][-90:]
    predictions = forecast['yhat'][-90:]
    mse = mean_squared_error(actuals, predictions)

    # Prepare the plotly figure for predictions
    figure = {
        'data': [
            {'x': forecast['ds'], 'y': forecast['yhat'], 'type': 'line', 'name': 'Predicted'},
            {'x': filtered_df['ds'], 'y': filtered_df['y'], 'type': 'line', 'name': 'Actual'},
        ],
        'layout': {
            'title': 'Predictions vs Actuals',
            'xaxis': {'title': 'Date'},
            'yaxis': {'title': 'WAIT TIME'},
        }
    }

    # Heatmap Data Preparation
    filtered_df_15['DATE'] = pd.to_datetime(filtered_df_15['DATE'])  # Ensure 'ds' is datetime for dayofweek
    filtered_df_15['day_of_week'] = filtered_df_15['DATE'].dt.dayofweek
    filtered_df_15['day_of_week'] = filtered_df_15['day_of_week'].sort_values()
    filtered_df_15['day_of_week_name'] = filtered_df_15['DATE'].dt.day_name()

    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    heatmap_df = filtered_df_15.pivot_table(index='day_of_week_name', columns='HOUR START OF 1 RIDE', values='WAIT TIME', aggfunc=np.mean)
    
    # Réordonnez l'index de heatmap_df selon l'ordre des jours de la semaine
    heatmap_df = heatmap_df.reindex(days_order)

    heatmap_fig = px.imshow(heatmap_df, labels=dict(x="Hour of Start", y="Day of Week", color="Average Wait Time"),
                            aspect="auto", color_continuous_scale='Viridis')
    heatmap_fig.update_layout(title='Average Wait Time Heatmap')

    # Return the figures and MSE text
    return figure, f'MSE: {mse}', heatmap_fig

if __name__ == '__main__':
    app.run_server(debug=True)
"""

'import dash\nfrom dash import dcc, html\nfrom dash.dependencies import Input, Output\nimport plotly.graph_objs as go\nimport pandas as pd\nimport numpy as np\nimport plotly.express as px\nfrom prophet import Prophet\nfrom sklearn.metrics import mean_squared_error\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom tqdm import tqdm\ntqdm.pandas()\nimport xgboost as xgb\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error, mean_absolute_percentage_error\nimport numpy as np\nfrom sklearn.preprocessing import StandardScaler\nfrom statsmodels.graphics.tsaplots import plot_acf, acf\nfrom sklearn.ensemble import RandomForestRegressor\nimport statsmodels.api as sm\n\ndf = pd.read_csv(\'data/final_data_day.csv\')\ndf_15 = pd.read_csv(\'data/final_data_15min.csv\')\n\n# Assuming df_15_bis is your DataFrame\navailable_attractions = df_15_bis[\'ATTRACTION NAME\'].unique()\n\napp = dash.Dash(__name__)\n\n# Set a white background and use exte

In [9]:
rush_hour_df = df_15_bis[(df_15_bis['HOUR START OF 1 RIDE'] > 9) & (df_15_bis['HOUR START OF 1 RIDE'] < 17)]
capacity_analysis = rush_hour_df.groupby('ATTRACTION NAME')['CAPACITY'].mean().reset_index()
capacity_analysis.sort_values(by = 'CAPACITY', inplace = True)

ratio_analysis = rush_hour_df.groupby('ATTRACTION NAME')['Ratio People/Capacity'].mean().reset_index()

from adjustText import adjust_text
from sklearn.linear_model import LinearRegression


attraction_names = combined_analysis['ATTRACTION NAME']
ratio_people_capacity = combined_analysis['Ratio People/Capacity']
wait_time = combined_analysis['WAIT TIME']

# Scatter plot with bubble size representing WAIT TIME
plt.figure(figsize=(10, 8))
scatter = plt.scatter(ratio_people_capacity, wait_time, c=wait_time, cmap='viridis', s=wait_time*2, alpha=0.7)

regression_model = LinearRegression()
X = ratio_people_capacity.values.reshape(-1, 1)
y = wait_time.values
regression_model.fit(X, y)
regression_line = regression_model.predict(X)

# Plot the linear regression line
plt.plot(ratio_people_capacity, regression_line, color='blue', linewidth=2, label='Linear Regression Line')

#plt.xlim(0, 0.9)
plt.title('Pourcentage of Capacity and Wait Time during Peak Hours')
plt.xlabel('Pourcentage of Capacity ')
plt.ylabel('Wait Time')

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Wait Time')

texts = [plt.text(ratio_people_capacity[i], wait_time[i], txt, fontsize=8, ha='center', va='center') for i, txt in enumerate(attraction_names)]

# Use adjust_text to automatically adjust label positions
adjust_text(texts)

plt.show()

ratio_analysis.sort_values(by = 'Ratio People/Capacity', inplace = True)

average_wait_time_by_attraction = rush_hour_df.groupby('ATTRACTION NAME')['WAIT TIME'].mean().reset_index()
average_wait_time_by_attraction.sort_values(by = 'WAIT TIME', inplace = True)

data_frames = [ratio_analysis, capacity_analysis, average_wait_time_by_attraction]
combined_analysis = reduce(lambda left, right: pd.merge(left, right, on='ATTRACTION NAME'), data_frames)

combined_analysis.sort_values('WAIT TIME', ascending= False)

KeyError: 'Column not found: Ratio People/Capacity'

In [11]:
from functools import reduce
from sklearn.linear_model import LinearRegression
# Assuming df_15_bis is already prepared
rush_hour_df = df_15_bis[(df_15_bis['HOUR START OF 1 RIDE'] > 9) & (df_15_bis['HOUR START OF 1 RIDE'] < 17)]
rush_hour_df['Ratio People/Capacity'] = rush_hour_df['ATTENDANCE'] / rush_hour_df['CAPACITY']
rush_hour_df.dropna()
capacity_analysis = rush_hour_df.groupby('ATTRACTION NAME')['CAPACITY'].mean().reset_index()
ratio_analysis = rush_hour_df.groupby('ATTRACTION NAME')['Ratio People/Capacity'].mean().reset_index()
average_wait_time_by_attraction = rush_hour_df.groupby('ATTRACTION NAME')['WAIT TIME'].mean().reset_index()

# Combine data frames
data_frames = [ratio_analysis, capacity_analysis, average_wait_time_by_attraction]
combined_analysis = reduce(lambda left, right: pd.merge(left, right, on='ATTRACTION NAME'), data_frames)
combined_analysis.sort_values('WAIT TIME', ascending=False)
# Créez le DataFrame `combined_analysis` si ce n'est pas déjà fait
combined_analysis['Average Wait Time'] = rush_hour_df.groupby('ATTRACTION NAME')['WAIT TIME'].mean().values


/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_17313/4226650655.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rush_hour_df['Ratio People/Capacity'] = rush_hour_df['ATTENDANCE'] / rush_hour_df['CAPACITY']


In [13]:
rush_hour_df.dtypes

DATE                                     datetime64[ns]
HOUR START OF 1 RIDE                              int64
ATTRACTION NAME                                  object
WAIT TIME                                         int64
NB UNITS                                        float64
GUEST CARRIED                                   float64
CAPACITY                                        float64
ADJUST CAPACITY                                 float64
OPEN_TIME                                         int64
REAL TIME ATTRACTION OPEN FOR 15MIN               int64
REAL TIME ATTRACTION CLOSES FOR 15MIN             int64
NB MAX UNITS                                    float64
TEMPERATURE                                     float64
TEMPERATURE FELT                                float64
TEMPERATURE MINIMUM                             float64
TEMPERATURE MAXIMUM                             float64
PRESSURE                                        float64
HUMIDITY                                        

In [14]:
ratio = (df['WAIT TIME'].sum() / df['ATTENDANCE'].sum()) * 10000
    

In [18]:
ratio * 10000

5.198420533137184